In [8]:
import numpy as np
import sys

class NeuralNetworks(object):
    '''Первый слой 3 входа, второй слой 2 скрытых нейрона, на выходе 1'''
    
    def __init__(self, learning_rate=0.1):
        self.weight_0_1 = np.random.normal(0.0, 0.1, (2, 3))  # Cлучайные веса от нулевого слоя к первому (выборки из нормального (гауссовского) распределения) в виде матрицы 2 х 3.
        self.weight_1_2 = np.random.normal(0.0, 0.1, (1, 2)) # Cлучайные веса от нулевого слоя к первому (выборки из нормального (гауссовского) распределения) в виде матрицы 1 х 2.
        self.sigmoid_mapper = np.vectorize(self.sigmoid)   # Возвращает результат функции sigmoid в виде вектора
        self.learning_rate = np.array([learning_rate])
        
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
    
    def predict(self, inputs):
        inputs_1 = np.dot(self.weight_0_1, inputs)
        outputs_1 = self.sigmoid_mapper(inputs_1)
        
        inputs_2 = np.dot(self.weight_1_2, outputs_1)
        outputs_2 = self.sigmoid_mapper(inputs_2)    
        return outputs_2
    
    def train(self, inputs, expected_predict):
        inputs_1 = np.dot(self.weight_0_1, inputs)
        outputs_1 = self.sigmoid_mapper(inputs_1)
                
        inputs_2 = np.dot(self.weight_1_2, outputs_1)
        outputs_2 = self.sigmoid_mapper(inputs_2)   
        actual_predict = outputs_2[0]
        
        error_layer_2 = np.array([actual_predict - expected_predict])  # sigmoid(z_j) - Y_j
        gradient_layer_2 = actual_predict * (1 - actual_predict )  
        # Градиен (вектор) характеризует направление возрастаниея функции для каждого a_j и равен sigmoid(z_j) * (1 - sigmoid(z_j)) или  sigmoid'(z_j)
        weights_delta_layer_2 = error_layer_2 * gradient_layer_2  #  Вектор ошибок нейронов выходного слоя (sigmoid(z_j) - Y_j) * sigmoid'(z_j)
        self.weight_1_2 = self.weight_1_2 - (np.dot(weights_delta_layer_2, outputs_1.reshape(1, (len(outputs_1))))) * self.learning_rate
        
        error_layer_1 = weights_delta_layer_2 * self.weight_1_2
        gradient_layer_1 = outputs_1 * (1 - outputs_1 )  
        weights_delta_layer_1 = error_layer_1 * gradient_layer_1
        self.weight_0_1 = self.weight_0_1 - np.dot(inputs.reshape(len(inputs), 1), weights_delta_layer_1).T * self.learning_rate

In [9]:
epochs = 5000
learning_rate = 0.05

my_nn = NeuralNetworks(learning_rate=learning_rate)

print(my_nn.weight_0_1)
print()
print(my_nn.weight_1_2)


[[ 0.1156405  -0.02288622  0.10349543]
 [ 0.08822605  0.0204991  -0.05816533]]

[[0.21710029 0.02863896]]


In [10]:
def MSE(y, Y):
    return np.mean((y - Y) ** 2)

train = [
    ([0, 0, 0], 0),
    ([0, 0, 1], 1),    
    ([0, 1, 0], 0),    
    ([0, 1, 1], 0),    
    ([1, 0, 0], 1),    
    ([1, 0, 1], 1),    
    ([1, 1, 0], 0),    
    ([1, 1, 1], 1)]

losses = {"train": [], 'validation': []}
for e in range(epochs):
    inputs_ = []
    correct_predictions = []
    for inputs_stat, correct_predict in train:
        my_nn.train(np.array(inputs_stat), correct_predict)
        inputs_.append(np.array(inputs_stat))
        correct_predictions.append(np.array(correct_predict))
        
    train_loss = MSE(my_nn.predict(np.array(inputs_).T), np.array(correct_predictions))
    sys.stdout.write('\rProgress: {}, Trainimg loss: {}'.format(str(100 * e/float(epochs))[:4], str(train_loss)[:5]))


Progress: 99.9, Trainimg loss: 0.003

In [11]:
for input_stat, correct_predict in train:
    print("For input: {} the predictions is: {}, excpected: {}".format(
        str(input_stat),
        str(my_nn.predict(np.array(input_stat)) > .5),
        str(correct_predict == 1)))

For input: [0, 0, 0] the predictions is: [False], excpected: False
For input: [0, 0, 1] the predictions is: [ True], excpected: True
For input: [0, 1, 0] the predictions is: [False], excpected: False
For input: [0, 1, 1] the predictions is: [False], excpected: False
For input: [1, 0, 0] the predictions is: [ True], excpected: True
For input: [1, 0, 1] the predictions is: [ True], excpected: True
For input: [1, 1, 0] the predictions is: [False], excpected: False
For input: [1, 1, 1] the predictions is: [ True], excpected: True


In [12]:
for input_stat, correct_predict in train:
    print("For input: {} the predictions is: {}, excpected: {}".format(
        str(input_stat),
        str(my_nn.predict(np.array(input_stat))),
        str(correct_predict == 1)))

For input: [0, 0, 0] the predictions is: [0.12293133], excpected: False
For input: [0, 0, 1] the predictions is: [0.94743299], excpected: True
For input: [0, 1, 0] the predictions is: [0.00084417], excpected: False
For input: [0, 1, 1] the predictions is: [0.03997568], excpected: False
For input: [1, 0, 0] the predictions is: [0.94753088], excpected: True
For input: [1, 0, 1] the predictions is: [0.97390378], excpected: True
For input: [1, 1, 0] the predictions is: [0.03998976], excpected: False
For input: [1, 1, 1] the predictions is: [0.92533532], excpected: True


In [13]:
my_nn.weight_0_1

array([[ 2.33658406, -2.59492596,  2.31230769],
       [-2.7261177 ,  3.25069822, -2.73824291]])

In [14]:
my_nn.weight_1_2

array([[ 3.68659663, -7.6165153 ]])